In [1]:
import sys
import os
sys.path.append('../src/')
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

import wavy

/Users/rodrigonader/anaconda3/envs/tf-metal/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_PATH = f"{os.getcwd()}/"
df = pd.read_csv(f"{DATA_PATH}../data/petr4.csv", index_col=0)
df = df.iloc[::-1]

In [4]:
# df = df[['price', 'timeTrade']]
df.reset_index(drop=True, inplace=True)

In [5]:
df.head(2)

,symbol,price,open,high,low,previous,quantityTrades,volumeAmount,volumeFinancier,timeTrade
0,petr4,16.58,16.60,16.60,16.58,16.60,2.0,500.0,9762.0,"Dec 28, 2012 7:29:00 PM"
1,petr4,16.60,16.59,16.63,16.59,16.57,7.0,7300.0,142639.0,"Dec 28, 2012 7:28:00 PM"


In [7]:
df = df[['price']].tail(10000)

In [8]:
df.head(2)

,price
1005428,28.96
1005429,28.96


In [9]:
# df = df.pct_change()
# print(len(df) - len(df.dropna()))
# df.dropna(inplace=True)

# scaler = StandardScaler()
# df[df.columns] = scaler.fit_transform(df)

In [23]:
%%time
x, y = wavy.create_panels(df, lookback=120, horizon=30)

CPU times: user 493 ms, sys: 8.49 ms, total: 502 ms
Wall time: 499 ms


In [24]:
x[0].head()

,price
1005428,28.96
1005429,28.96
1005430,28.96
1005431,28.94
1005432,28.94


In [25]:
xmax = x.max()

In [26]:
x[0]

,price
0,29.0


In [18]:
%%time
ymax = y.max()

CPU times: user 1.1 s, sys: 25 ms, total: 1.12 s
Wall time: 1.11 s


In [20]:
# xlast = x.iloc[-1]
xmax = x.max()

In [21]:
x[0]

,price
0,29.0


In [13]:
ylabels = ymax > xmax

In [14]:
ylabels.sample(10000, how='spaced').as_dataframe().price.value_counts(normalize=True)

False    0.6637
True     0.3363
Name: price, dtype: float64

In [15]:
x[0]

,price
0,29.0


In [ ]:
baseline = wavy.models.BaselineConstant(x, y, model_type="regression", constant=0)

baseline.fit()
baseline.score()

In [ ]:
log = wavy.models.LogisticRegression(x, ymax)
log.fit(epochs=10, verbose=True)
log.score()

In [ ]:
dense = wavy.models.DenseModel(x, ymax, 'classification')
dense.fit(epochs=10, verbose=True)
dense.score()

In [ ]:
conv = wavy.models.ConvModel(x, ymax, 'classification')
conv.fit(epochs=10, verbose=True)
conv.score()

In [ ]:
residuals = conv.residuals()

In [ ]:
len(residuals)

In [ ]:
residuals.sample(10000, how='spaced').plot()